In [1]:
from hetbuilder.algorithm import CoincidenceAlgorithm
from hetbuilder.plotting import InteractivePlot
from ase.io.vasp import read_vasp, write_vasp

In [2]:
bottom = read_vasp('CONTCAR_SiO_film')
top = read_vasp('CONTCAR_FCC_film')
# we set up the algorithm class
alg = CoincidenceAlgorithm(bottom, top)
# we run the algorithm for a choice of parameters
results = alg.run(Nmax = 5, Nmin = 0, 
                  tolerance = 2, weight = 0.5)

WARNING  It seems that the structure Atoms(symbols='H4O12Si5', pbc=True, cell=[[3.6603421357432016,                
         3.660342135743201, 5e-16], [-3.6603421357432, 3.660342135743201, 0.0], [0.0, 0.0, 50.0]]) is not          
         primitive.

WARNING  This might lead to unexpected results.

INFO     Searching 91 angles between 0.0 and 90.0 degree with a stepsize of 1.0 degree.

INFO     Using 12 OpenMP threads.

INFO     Running through 910000 grid points...

INFO     Found 40 results.

In [3]:
#for j in results:
#    print(j)

In [8]:
#%matplotlib widget
import matplotlib
matplotlib.use('Qt5Agg') # if failed,  pip install PyQt5  PySide2
iplot = InteractivePlot(bottom=bottom, top=top, results=results, weight=1)
iplot.plot_results()

Saved structure to H4Au24O12Si5_8.0000_degree_2.vasp with M = [1 0 // 0 1], N = [2 1 // 0 2], stress = 0.0595, strain = 1.6295
Saved structure to H4Au18O12Si5_0.0000_degree_1.vasp with M = [1 0 // 0 1], N = [2 1 // -1 1], stress = 0.1631, strain = 0.5589
Saved structure to H4Au18O12Si5_0.0000_degree_1.vasp with M = [1 0 // 0 1], N = [2 1 // -1 1], stress = 0.1631, strain = 0.5589


In [5]:
j = results[1]
print(type(j))
print(j.bottom)
print(j.top)
print(j.stack)
print(j.strain)
# view(bottom)
# view(j.stack)


<class 'hetbuilder.algorithm.Interface'>
Atoms(symbols='H4O12Si5', pbc=True, cell=[[3.6603421357432016, 3.660342135743201, 0.0], [-3.6603421357432, 3.660342135743201, 0.0], [0.0, 0.0, 121.95699699847876]], initial_magmoms=...)
Atoms(symbols='Au18', pbc=True, cell=[[4.406014405277883, 2.5438136029405545, 0.0], [-4.406014405277883, 2.5438136029405545, 0.0], [0.0, 0.0, 111.991652324759]], initial_magmoms=...)
Atoms(symbols='H4O12Si5Au18', pbc=True, cell=[[4.033178270510542, 3.102077869341878, 0.0], [-4.0331782705105415, 3.102077869341878, 0.0], [0.0, 0.0, 52.94864932323776]], initial_magmoms=...)
0.5589439202227763


In [6]:
from ase import Atoms
from ase.build import make_supercell
from ase.build.tools import sort
from numpy import array, zeros, concatenate
from ase.visualize import view

def build_supercells(primitive_bottom: Atoms = None, 
                         primitive_top: Atoms = None,
                         M: array = None,
                         N: array = None,
                         angle_z: float = None, 
                         weight: float = 0.5,
                         distance: float = 3.5,
                         vacuum: float = 15,
                         pbc: list = [True, True, False], 
                         reorder=True ) -> Atoms:
    """
    For construct supercell for two primitive cell known transition matrix and rotation angle of top layer around z-dir\n
    Input: must be primitive cell\n
           M, N: bottom, top matrix\n
           angle_z: units: degree, +z direction\n
           distance, vacuum: units: Angstron\n
           weight: must be 0~1, fixing bottom~top\n
           pbc, reorder: the default value is good\n
    Output: 
           heterostructure
    Usage:
           For output of hetbuilder, results is a list contained many structures.
           defined: j = results[0] or any index
                    build_supercells(j.bottom, j.top, j.M, j.N, j.angle, j._weight) # note: _weight
           For general situation,
                    build_supercells(bottom, top, M, N, angle, weight)
    """
    bottom = primitive_bottom.copy()
    top = primitive_top.copy()
    #print(top)
    # make_supercell
    bottom_sup = make_supercell(bottom, M)
    top_sup = make_supercell(top, N)
    #print(bottom_sup)

    # rotate top layer around z, units: degree
    top_sup.rotate(angle_z, 'z', rotate_cell=True)
    #print(top_sup)
    # translate from hetbuilder (atom_functions.cpp/.h) stack_atoms function
    stack = stack_atoms(bottom_sup, top_sup, weight, distance, vacuum, pbc, reorder)
    #print(1)
    return stack

def stack_atoms(bottom_sup: Atoms = None, top_sup: Atoms = None, weight: float = 0.5, distance: float = 3.5, vacuum: float = 15, 
                pbc: list = [True, True, False], reorder: bool=True, shift_tolerance: float = 1e-5) -> Atoms:
    """
    After searching the coincidenced supecell, we get the transition matrix(M for bottom, N for top), and make two supercells, and then try to match them.\n
    Additionally, bottom, top, weight, distance, vacuum.\n
    usage: updating...
    Noted: the cell C = A + weight * [B - A], A - bottom, B - top
    """
    # get the max, min z-position 
    bottom = bottom_sup.copy()
    top = top_sup.copy()
    min_z1, max_z1 = bottom.positions[:,2].min(), bottom.positions[:,2].max()
    min_z2, max_z2 = top.positions[:,2].min(), top.positions[:,2].max()
    bottom_thickness = max_z1 - min_z1
    top_thickness = max_z2 - min_z2
    #print(bottom_thickness)
    #print(distance)
    shift = bottom_thickness + distance

    # add distance
    bottom.positions[:,2] -= min_z1
    bottom.positions[:,2] += shift_tolerance
    top.positions[:,2] -= min_z2
    top.positions[:,2] += shift
    top.positions[:,2] += shift_tolerance

    # generate lattice
    new_lattice = zeros((3,3))
    
    # C = A + weight * [B - A]
    for i in range(2):  
        for j in range(2):
            #print(i,j)
            new_lattice[i, j] = bottom.cell[i, j] + weight * (top.cell[i, j] - bottom.cell[i, j])
    new_lattice[2,2] = bottom_thickness + distance + top_thickness + vacuum

    # combine the position and symbols information
    all_positions = concatenate([bottom.positions, top.positions], axis=0)
    all_symbols = bottom.get_chemical_symbols() + top.get_chemical_symbols()

    # create new Atoms
    stack = Atoms(symbols=all_symbols, positions=all_positions, cell=new_lattice, pbc=pbc)
    stack.center()
    if reorder:
        stack = sort(stack)
    return stack


def scale_cell_xy(atoms_origin: Atoms = None, new_cell: array = None):
    """
    After calculating C - supercell, try to scall xy of cell, but fixed z.\n
    It's important for fixing someone's lattice constant, otherwise, the z-dir constant'll changed!
    """
    atoms = atoms_origin.copy()
    # 获取原始的笛卡尔坐标
    original_cart_pos = atoms.get_positions()
    
    # 获取按新晶胞缩放的分数坐标
    scaled_positions = atoms.get_scaled_positions()

    # 更新晶胞，注意这里假设new_cell是一个numpy数组或类似的可直接赋值给atoms.cell的结构
    atoms.set_cell(new_cell)

    # 将缩放的分数坐标转换回笛卡尔坐标，这里直接使用Atoms对象的.set_scaled_positions方法
    atoms.set_scaled_positions(scaled_positions)

    # 保持z方向上的笛卡尔坐标不变
    updated_cart_pos = atoms.get_positions()
    updated_cart_pos[:, 2] = original_cart_pos[:, 2]

    # 更新原子的位置
    atoms.set_positions(updated_cart_pos)

print(build_supercells(bottom, top, j.M, j.N, j.angle, j._weight))
print(build_supercells(bottom, top, j.M, j.N, j.angle, 0.5))
print(build_supercells(bottom, top, j.M, j.N, j.angle, 0))
print(build_supercells(bottom, top, j.M, j.N, j.angle, 1))

Atoms(symbols='Au18H4O12Si5', pbc=[True, True, False], cell=[[4.033178270510542, 3.102077869341878, 0.0], [-4.0331782705105415, 3.102077869341878, 0.0], [0.0, 0.0, 52.44864932323775]])
Atoms(symbols='Au18H4O12Si5', pbc=[True, True, False], cell=[[4.033178270510542, 3.102077869341878, 0.0], [-4.0331782705105415, 3.102077869341878, 0.0], [0.0, 0.0, 52.44864932323775]])
Atoms(symbols='Au18H4O12Si5', pbc=[True, True, False], cell=[[3.6603421357432016, 3.660342135743201, 0.0], [-3.6603421357432, 3.660342135743201, 0.0], [0.0, 0.0, 52.44864932323775]])
Atoms(symbols='Au18H4O12Si5', pbc=[True, True, False], cell=[[4.406014405277883, 2.5438136029405545, 0.0], [-4.406014405277883, 2.5438136029405545, 0.0], [0.0, 0.0, 52.44864932323775]])


In [7]:
print(build_supercells(bottom, top, j.M, j.N, j.angle, 0.5))

Atoms(symbols='Au18H4O12Si5', pbc=[True, True, False], cell=[[4.033178270510542, 3.102077869341878, 0.0], [-4.0331782705105415, 3.102077869341878, 0.0], [0.0, 0.0, 52.44864932323775]])
